In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pandas_ta as ta
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats
from IPython.display import display

In [ ]:
aapl = yf.Ticker("NVDA")
df_apple = aapl.history(start="2021-1-1", end="2021-12-31", interval="1d")
df_apple.head()

In [ ]:
# Let's compute the 5-d, 15-d and 25-d SMA for visualization
df_apple["5d_sma_price"] = df_apple['Close'].rolling(5).mean()
df_apple["15d_sma_price"] = df_apple['Close'].rolling(15).mean()
df_apple["25d_sma_price"] = df_apple['Close'].rolling(25).mean()

# The 25-d SMA for trading volume
df_apple["25d_sma_volume"] = df_apple['Volume'].rolling(25).mean()
df_apple = df_apple[df_apple["25d_sma_price"].notna()]
df_apple.head()

In [ ]:
# Let's visualize the SMAs in a graph
plt.figure(figsize=(12, 6))
plt.plot(df_apple['Close'], color='blue', linewidth=0.5, label='Closing price')
plt.plot(df_apple['5d_sma_price'], color='black', linewidth=0.5, label='5-d SMA')
plt.plot(df_apple['15d_sma_price'], color='green', linewidth=0.5, label='15-d SMA')
plt.plot(df_apple['25d_sma_price'], color='red', linewidth=0.5, label='25-d SMA')
plt.title("5-d, 15-d & 25-d SMA of AAPL stock closing prices")
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
class TradeLogger(bt.analyzers.Analyzer):
    """
    Analyzer returning closed trades information.
    """

    def start(self):
        super(TradeLogger, self).start()

    def create_analysis(self):
        self.rets = []
        self.vals = dict()

    def notify_trade(self, trade):
        """Receives trade notifications before each next cycle"""
        if trade.isclosed:
            self.vals = {'Date': self.strategy.datetime.datetime(),
                         'Gross PnL': round(trade.pnl, 2),
                         'Net PnL': round(trade.pnlcomm, 2),
                         'Trade commission': trade.commission,
                         'Trade duration (in days)': (trade.dtclose - trade.dtopen)
            }
            self.rets.append(self.vals)

    def get_analysis(self):
        return self.rets


def set_and_run(data, strategy, startcash, commission, stake):
    # Initialize cerebro engine, add the strategy and initial capital
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy)
    cerebro.adddata(data)
    cerebro.broker.setcash(startcash)
    print("Starting Portfolio Value: {}".format(cerebro.broker.getvalue()))

    # Set the broker commission
    cerebro.broker.setcommission(commission) 

    # Number of shares to buy/sell
    cerebro.addsizer(bt.sizers.FixedSize, stake=stake)

    # Add evaluation metrics
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, compression=1, factor=365, annualize=True)
    cerebro.addanalyzer(TradeLogger, _name="trade_logger")

    results = cerebro.run()
    print("Final Portfolio Value: {}".format(cerebro.broker.getvalue()))
    print("Sharpe Ratio: {}".format(results[0].analyzers.sharperatio.get_analysis()['sharperatio']))
    #display(pd.DataFrame(results[0].analyzers.trade_logger.get_analysis()))

    plt.rcParams['figure.figsize'] = (16, 8)
    fig = cerebro.plot(barupfill=False,
                       bardownfill=False,
                       style='candle',
                       plotdist=0.5, 
                       volume=True,
                       barup='green',
                       valuetags=False,
                       subtxtsize=8)

    # Create the figure and plot the data
    plt.savefig('AAPL.svg', dpi=300)  

In [ ]:
class FirstStrategy(bt.Strategy):
    params = (('close_period', 5),
              ('volume_period', 4)
    )

    def __init__(self):
        self.close_sma = bt.indicators.SMA(self.data.close, period=self.params.close_period)
        self.volume_sma = bt.indicators.SMA(self.data.volume, period=self.params.volume_period)
        
    def next(self):
        if not self.position:
            if self.data.close < self.close_sma and self.data.volume > self.volume_sma:
                self.buy()
        else:
            if self.data.close > self.close_sma and self.data.volume < self.volume_sma: 
                self.sell()

In [ ]:
data = bt.feeds.PandasData(dataname=df_apple)

startcash = 100000
commission = 0.001
stake = 100
# Backtesting the model's parameters
cerebro = bt.Cerebro()
cerebro.adddata(data)
strats = cerebro.optstrategy(FirstStrategy,
                             close_period=range(2, 25), 
                             volume_period=range(2, 25))
cerebro.broker.setcash(startcash)
cerebro.broker.setcommission(commission) 
cerebro.addsizer(bt.sizers.FixedSize, stake=stake)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, factor=365)
cerebro.addanalyzer(bt.analyzers.VWR, timeframe=bt.TimeFrame.Days, tann=365)
results = cerebro.run()

df_result = pd.DataFrame([{"Close period": result[0].params.close_period, 
                           "Volume period": result[0].params.volume_period,
                           "Sharpe ratio": result[0].analyzers.sharperatio.get_analysis()['sharperatio'], 
                           "Variability weighted return (in %)": result[0].analyzers.vwr.get_analysis()['vwr']
                          } for result in results])
display(df_result.sort_values("Sharpe ratio", ascending=False))

In [ ]:
data = bt.feeds.PandasData(dataname=df_apple)
startcash = 100000
commission = 0.001
stake = 100
# Run the strategy & plot the results
set_and_run(data,
            FirstStrategy,
            startcash,
            commission,
            stake)

In [ ]:
class SecondStrategy(bt.Strategy):
    params = (('short_period', 6),
              ('long_period', 98)
    )

    def __init__(self):
        self.short_period_sma = bt.indicators.SMA(self.data.close, period=self.params.short_period)
        self.long_period_sma = bt.indicators.SMA(self.data.close, period=self.params.long_period)
        
    def next(self):
        if not self.position:
            if self.short_period_sma > self.long_period_sma:
                self.buy()
        else:
            if self.short_period_sma < self.long_period_sma:
                self.sell()

In [ ]:
# Backtesting the model's parameters
cerebro = bt.Cerebro()
cerebro.adddata(data)
strats = cerebro.optstrategy(SecondStrategy,
                             short_period=range(1, 30), 
                             long_period=range(80, 120))
cerebro.broker.setcash(startcash)
cerebro.broker.setcommission(commission) 
cerebro.addsizer(bt.sizers.FixedSize, stake=stake)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, factor=365)
cerebro.addanalyzer(bt.analyzers.VWR, timeframe=bt.TimeFrame.Days, tann=365)
results = cerebro.run()

df_result = pd.DataFrame([{"Short period": result[0].params.short_period, 
                           "Long period": result[0].params.long_period,
                           "Sharpe ratio": result[0].analyzers.sharperatio.get_analysis()['sharperatio'], 
                           "Variability weighted return (in %)": result[0].analyzers.vwr.get_analysis()['vwr']
                          } for result in results])
display(df_result.sort_values("Sharpe ratio", ascending=False))

In [ ]:
# Run the strategy & plot the results
set_and_run(data,
            SecondStrategy,
            startcash,
            commission,
            stake)

In [ ]:
class MACDStrategy(bt.Strategy):
    params = (('short_period', 25),
              ('long_period', 51),
              ('signal_period', 9)
    )

    def __init__(self):
        self.macd = bt.indicators.MACD(self.data.close,
                                       period_me1=self.params.short_period,
                                       period_me2=self.params.long_period,
                                       period_signal=self.params.signal_period)
        
        self.macd_cross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal, plot=True, subplot=True)

    def next(self):
        if not self.position:
            if self.macd_cross > 0:
                self.buy()
        else:
            if self.macd_cross < 0:
                self.sell()

In [ ]:
# Backtesting the model's parameters
cerebro = bt.Cerebro()
cerebro.adddata(data)
strats = cerebro.optstrategy(MACDStrategy,
                             short_period=range(20, 30), 
                             long_period=range(50, 70))
cerebro.broker.setcash(startcash)
cerebro.broker.setcommission(commission) 
cerebro.addsizer(bt.sizers.FixedSize, stake=stake)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, factor=365)
cerebro.addanalyzer(bt.analyzers.VWR, timeframe=bt.TimeFrame.Days, tann=365)
results = cerebro.run()

df_result = pd.DataFrame([{"Short period": result[0].params.short_period, 
                           "Long period": result[0].params.long_period,
                           "Sharpe ratio": result[0].analyzers.sharperatio.get_analysis()['sharperatio'], 
                           "Variability weighted return (in %)": result[0].analyzers.vwr.get_analysis()['vwr']
                          } for result in results])
display(df_result.sort_values("Sharpe ratio", ascending=False))

In [ ]:
data = bt.feeds.PandasData(dataname=df_apple)
print(data.dataframe)
startcash = 100000
commission = 0.001
stake = 100
# Run the strategy & plot the results
set_and_run(data,
            MACDStrategy,
            startcash,
            commission,
            stake)

In [ ]:
class BollingerBandsStrategy(bt.Strategy):
    params = (('MBB_period', 17),
              ('devfactor', 3)
            )

    def __init__(self):
        self.boll = bt.indicators.BollingerBands(period=self.params.MBB_period,
                                                 devfactor=self.params.devfactor)

    def next(self):
        if not self.position:
          if self.data.close < self.boll.lines.bot:
             self.buy()
        else:
          if self.data.close > self.boll.lines.top:
            self.sell()

In [ ]:
# Backtesting the MBB period and deviation factor
cerebro = bt.Cerebro()
cerebro.adddata(data)
strats = cerebro.optstrategy(BollingerBandsStrategy,
                             MBB_period=range(5, 30), 
                             devfactor=range(2, 7))
cerebro.broker.setcash(startcash)
cerebro.broker.setcommission(commission) 
cerebro.addsizer(bt.sizers.FixedSize, stake=stake)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, factor=365)
cerebro.addanalyzer(bt.analyzers.VWR, timeframe=bt.TimeFrame.Days, tann=365)
results = cerebro.run()

df_result = pd.DataFrame([{"MBB period": result[0].params.MBB_period, 
                           "Deviation factor": result[0].params.devfactor,
                           "Sharpe ratio": result[0].analyzers.sharperatio.get_analysis()['sharperatio'], 
                           "Variability weighted return (in %)": result[0].analyzers.vwr.get_analysis()['vwr']
                          } for result in results])
display(df_result.sort_values("Sharpe ratio", ascending=False))

In [ ]:
# Run the strategy & plot the results
set_and_run(data,
            BollingerBandsStrategy,
            startcash,
            commission,
            stake)

In [ ]:
class SecondStrategyStopLoss(bt.Strategy):
    params = (('short_period', 10),
              ('long_period', 20),
              ('stop_loss', 0.02),  # price is 2% less than the entry point
              ('trail', True)      # controls whether the stop-loss is dynamic or not
    )

    def __init__(self):
        self.short_period_sma = bt.indicators.SMA(self.data.close, period=self.params.short_period)
        self.long_period_sma = bt.indicators.SMA(self.data.close, period=self.params.long_period)
        self.crossup = bt.indicators.CrossUp(self.short_period_sma, self.long_period_sma)

    def notify_order(self, order):
        if not order.status == order.Completed:
            return  # discard any other notification

        if not self.position:  # we left the market
            print('SELL @price: {:.2f}'.format(order.executed.price))
            return

        # We have entered the market
        print('BUY @price: {:.2f}'.format(order.executed.price))
        
    def next(self):
        if not self.position and self.crossup > 0:
            self.buy()
        
        if self.position:
            if not self.params.trail:
                stop_price = self.data.close * (1.0 - self.params.stop_loss)
                self.sell(exectype=bt.Order.Stop, price=stop_price)
            else:
                self.sell(exectype=bt.Order.StopTrail,
                          trailamount=self.params.trail)

In [ ]:
# Run the strategy & plot the results
set_and_run(data,
            SecondStrategyStopLoss,
            startcash,
            commission,
            stake)

In [ ]:
class DonchianChannels(bt.Indicator):
    '''
    Params Note:
      - `lookback` (default: -1)
        If `-1`, the bars to consider will start 1 bar in the past and the
        current high/low may break through the channel.
        If `0`, the current prices will be considered for the Donchian
        Channel. This means that the price will **NEVER** break through the
        upper/lower channel bands.
    '''

    alias = ('DCH', 'DonchianChannel',)

    lines = ('dcm', 'dch', 'dcl',)  # dc middle, dc high, dc low
    params = dict(
        period=20,
        lookback=-1,  # consider current bar or not
    )

    plotinfo = dict(subplot=False)  # plot along with data
    plotlines = dict(
        dcm=dict(ls='--'),  # dashed line
        dch=dict(_samecolor=True),  # use same color as prev line (dcm)
        dcl=dict(_samecolor=True),  # use same color as prev line (dch)
    )

    def __init__(self):
        hi, lo = self.data.high, self.data.low
        if self.p.lookback:  # move backwards as needed
            hi, lo = hi(self.p.lookback), lo(self.p.lookback)

        self.l.dch = bt.ind.Highest(hi, period=self.p.period)
        self.l.dcl = bt.ind.Lowest(lo, period=self.p.period)
        self.l.dcm = (self.l.dch + self.l.dcl) / 2.0  # avg of the above

class DonChainStrategy(bt.Strategy):
    params = (('lower_length', 20),
              ('upper_length', 20)
            )

    def __init__(self):
        self.donchain = DonchianChannels(self.data, lower_length=self.params.lower_length,
                                                 upper_length=self.params.upper_length)

    def next(self):
        if not self.position:
          if self.data.close == self.donchain.low | self.data.low == self.donchain.low:
             self.buy()
        else:
          if self.data.close == self.donchain.high | self.data.high == self.donchain.high:
            self.sell()

In [ ]:
# Run the strategy & plot the results
set_and_run(data,
            DonChainStrategy,
            startcash,
            commission,
            stake)